# Blocks and Other Pyomo Best Practices

## Learning Objectives

1. Practice using blocks to organize Pyomo models with a hierachical structure (e.g., two-stage stochastic program)
2. Learn best practices for organizing Pyomo models and data

## Blocks in Pyomo

Reference: [Chapter 8: Stuctured Modeling with Blocks](https://link.springer.com/chapter/10.1007/978-3-030-68928-5_8), *Pyomo -- Optimization Modeling in Python*, Bynum et al. (2021).

Blocks provide a convenient way to expressed hierachically-structured models in Pyomo. As an example, consider the special structure in the *Farmer's* example for [](./SP.ipynb).

Asking ChatGPT to rewrite our model from [](./SP.ipynb) with blocks gives the following code (which was then slightly modified):

In [1]:
from pyomo.environ import ConcreteModel, Var, Objective, Block, Constraint, NonNegativeReals, summation, SolverFactory, minimize

def build_sp_model(yields):
    '''
    Rewritten version of the stochastic programming model using blocks.
    
    Arguments:
        yields: Yield information as a list, following the rank [wheat, corn, beets]
        
    Return: 
        model: farmer problem model using blocks
    '''
    
    # Model
    model = ConcreteModel()

    # Sets
    all_crops = ["WHEAT", "CORN", "BEETS"]
    purchase_crops = ["WHEAT", "CORN"]
    sell_crops = ["WHEAT", "CORN", "BEETS_FAVORABLE", "BEETS_UNFAVORABLE"]
    scenarios = ["ABOVE", "AVERAGE", "BELOW"]

    # First-stage decision: how much to plant for each crop
    model.X = Var(all_crops, within=NonNegativeReals)

    # Define a block for each scenario
    def scenario_block_rule(b, scenario):
        # Second-stage decision variables
        b.Y = Var(purchase_crops, within=NonNegativeReals)  # How much to purchase in this scenario
        b.W = Var(sell_crops, within=NonNegativeReals)  # How much to sell in this scenario
        
        # Purchase cost and sales revenue for this scenario
        if scenario == "ABOVE":
            b.purchase_cost = 238 * b.Y["WHEAT"] + 210 * b.Y["CORN"]
            b.sales_revenue = (
                170 * b.W["WHEAT"] + 150 * b.W["CORN"] 
                + 36 * b.W["BEETS_FAVORABLE"] + 10 * b.W["BEETS_UNFAVORABLE"]
            )
        elif scenario == "AVERAGE":
            b.purchase_cost = 238 * b.Y["WHEAT"] + 210 * b.Y["CORN"]
            b.sales_revenue = (
                170 * b.W["WHEAT"] + 150 * b.W["CORN"] 
                + 36 * b.W["BEETS_FAVORABLE"] + 10 * b.W["BEETS_UNFAVORABLE"]
            )
        else:  # BELOW
            b.purchase_cost = 238 * b.Y["WHEAT"] + 210 * b.Y["CORN"]
            b.sales_revenue = (
                170 * b.W["WHEAT"] + 150 * b.W["CORN"] 
                + 36 * b.W["BEETS_FAVORABLE"] + 10 * b.W["BEETS_UNFAVORABLE"]
            )

        # Scenario constraints
        if scenario == "ABOVE":
            b.wheat_constraint = Constraint(expr=yields[0] * 1.2 * model.X["WHEAT"] + b.Y["WHEAT"] - b.W["WHEAT"] >= 200)
            b.corn_constraint = Constraint(expr=yields[1] * 1.2 * model.X["CORN"] + b.Y["CORN"] - b.W["CORN"] >= 240)
            b.beets_constraint = Constraint(expr=yields[2] * 1.2 * model.X["BEETS"] 
                                            - b.W["BEETS_FAVORABLE"] - b.W["BEETS_UNFAVORABLE"] >= 0)
        elif scenario == "AVERAGE":
            b.wheat_constraint = Constraint(expr=yields[0] * model.X["WHEAT"] + b.Y["WHEAT"] - b.W["WHEAT"] >= 200)
            b.corn_constraint = Constraint(expr=yields[1] * model.X["CORN"] + b.Y["CORN"] - b.W["CORN"] >= 240)
            b.beets_constraint = Constraint(expr=yields[2] * model.X["BEETS"] 
                                            - b.W["BEETS_FAVORABLE"] - b.W["BEETS_UNFAVORABLE"] >= 0)
        else:  # BELOW
            b.wheat_constraint = Constraint(expr=yields[0] * 0.8 * model.X["WHEAT"] + b.Y["WHEAT"] - b.W["WHEAT"] >= 200)
            b.corn_constraint = Constraint(expr=yields[1] * 0.8 * model.X["CORN"] + b.Y["CORN"] - b.W["CORN"] >= 240)
            b.beets_constraint = Constraint(expr=yields[2] * 0.8 * model.X["BEETS"] 
                                            - b.W["BEETS_FAVORABLE"] - b.W["BEETS_UNFAVORABLE"] >= 0)

        # Set upper bounds for BEETS_FAVORABLE
        b.W["BEETS_FAVORABLE"].setub(6000)

    # Create blocks for each scenario
    model.scenarios = Block(scenarios, rule=scenario_block_rule)

    # Objective function
    def objective_rule(m):
        planting_cost = 150 * m.X["WHEAT"] + 230 * m.X["CORN"] + 260 * m.X["BEETS"]
        expected_purchase_cost = (1/3) * sum(m.scenarios[sc].purchase_cost for sc in scenarios)
        expected_sales_revenue = (1/3) * sum(m.scenarios[sc].sales_revenue for sc in scenarios)
        return planting_cost + expected_purchase_cost - expected_sales_revenue

    model.OBJ = Objective(rule=objective_rule, sense=minimize)

    # First-stage constraint: total area allocated to crops should not exceed 500
    model.total_land_constraint = Constraint(expr=summation(model.X) <= 500)

    return model

yields = [2.5, 3.0, 20.0]

model = build_sp_model(yields)
solver = SolverFactory('cbc')
solver.solve(model)

# Display the results
print("Planting decisions:")
for crop in model.X:
    print(f"{crop}: {model.X[crop]()}")

Planting decisions:
WHEAT: 170.0
CORN: 80.0
BEETS: 250.0


We got the answer answer as [](./SP.ipynb)!

## Seperate the Data from the Model

The above blocks example is not really a big improvement from our alternate implementation in [](./SP.ipynb).

In the above code (from ChatGPT), we see:
* Data for each scenario are hardcoded into the model
* If statements are used to toggle the correct constraints for each scenario

The above code is not general purpose; updating it to use alternate scenario data or consider additional crops would take a lot of manual effort. It would be easy to make mistakes.

Thus, it is **best practice** to always **seperate your specific problem data** from the **general mathematical model**. Let's see an example.

### Pandas DataFrames

Let's use a pandas dataframe to store our problem data.

In [2]:
import pandas as pd

nominal_data = pd.read_csv("../data/farmers.csv")

nominal_data.head()

,Unnamed: 0,Wheat,Corn,Favorable_Beets,Regular_Beets,Units
0,Yield,2.5,3.0,20.0,20.0,T/acre
1,Planting Cost,150.0,230.0,260.0,260.0,USD/acre
2,Selling Price,170.0,150.0,36.0,10.0,USD/T
3,Purchase Price,238.0,210.0,NaN,NaN,USD/T
4,Minimum Requirement,200.0,240.0,NaN,NaN,T


This looks great. But it is best practice to have the rows be instances of data and the columns to be the types of data. We need to transpose the CSV file. Also, let's drop the units for simplicity. ChatGPT is actually really helpful for transposing the CSV file!

In [10]:
nominal_data = pd.read_csv("../data/farmers2.csv")
nominal_data.head()

,index,Yield,Planting Cost,Selling Price,Purchase Price,Minimum Requirement,Maximum Production
0,Wheat,2.5,150.0,170.0,238.0,200.0,NaN
1,Corn,3.0,230.0,150.0,210.0,240.0,NaN
2,Favorable_Beets,20.0,260.0,36.0,NaN,NaN,6000.0
3,Regular_Beets,20.0,260.0,10.0,NaN,NaN,NaN
4,Units,T/acre,USD/acre,USD/T,USD/T,T,T


Now let's drop the units for simplicity.

In [11]:
nominal_data = nominal_data.set_index("index")
nominal_data.head()

,Yield,Planting Cost,Selling Price,Purchase Price,Minimum Requirement,Maximum Production
index,,,,,,
Wheat,2.5,150.0,170.0,238.0,200.0,NaN
Corn,3.0,230.0,150.0,210.0,240.0,NaN
Favorable_Beets,20.0,260.0,36.0,NaN,NaN,6000.0
Regular_Beets,20.0,260.0,10.0,NaN,NaN,NaN
Units,T/acre,USD/acre,USD/T,USD/T,T,T


In [12]:
nominal_data.drop("Units", inplace=True)
nominal_data.head()


,Yield,Planting Cost,Selling Price,Purchase Price,Minimum Requirement,Maximum Production
index,,,,,,
Wheat,2.5,150.0,170.0,238.0,200.0,NaN
Corn,3.0,230.0,150.0,210.0,240.0,NaN
Favorable_Beets,20.0,260.0,36.0,NaN,NaN,6000.0
Regular_Beets,20.0,260.0,10.0,NaN,NaN,NaN


Finally, we need to convert the data entries from strings to numbers.

In [22]:
# Convert all elements to numbers
nominal_data = nominal_data.map(lambda x: pd.to_numeric(x, errors='coerce'))

/var/folders/3w/vr4xmyqs451dg23xk88pqcg00000gq/T/ipykernel_36626/3069360891.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  nominal_data = nominal_data.applymap(lambda x: pd.to_numeric(x, errors='coerce'))


In [23]:
nominal_data.head()

,Yield,Planting Cost,Selling Price,Purchase Price,Minimum Requirement,Maximum Production
index,,,,,,
Wheat,2.5,150.0,170.0,238.0,200.0,NaN
Corn,3.0,230.0,150.0,210.0,240.0,NaN
Favorable_Beets,20.0,260.0,36.0,NaN,NaN,6000.0
Regular_Beets,20.0,260.0,10.0,NaN,NaN,NaN


### Single Scenario Optimization Problem

In [28]:
from pyomo.environ import Set, Param, Any

# Model
m = ConcreteModel()

# Sets
crops = nominal_data.index.to_list()
m.CROPS = Set(initialize=crops)

# Parameters
# TODO: Replace "Any" with something more appropriate; need to ask Pyomo team
m.cost = Param(m.CROPS, initialize=nominal_data["Planting Cost"], within=Any)
m.sell_price = Param(m.CROPS, initialize=nominal_data["Selling Price"], within=Any)
m.purchase_price = Param(m.CROPS, initialize=nominal_data["Purchase Price"], within=Any)
m.crop_yield = Param(m.CROPS, initialize=nominal_data["Yield"], within=Any)
m.min_required = Param(m.CROPS, initialize=nominal_data["Minimum Requirement"], within=Any)
m.max_possible = Param(m.CROPS, initialize=nominal_data["Maximum Production"], within=Any)

# Stage 1 decision variables
m.x = Var(m.CROPS, within=NonNegativeReals)

# Stage 2 decision variables
m.y = Var(m.CROPS, within=NonNegativeReals)
m.w = Var(m.CROPS, within=NonNegativeReals)

# Total area constraint
@m.Constraint()
def total_area(m):
    return sum(m.x[crop] for crop in m.CROPS) <= 500

# Total 

1 Set Declarations
    CROPS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'Wheat', 'Corn', 'Favorable_Beets', 'Regular_Beets'}

6 Param Declarations
    cost : Size=4, Index=CROPS, Domain=Any, Default=None, Mutable=False
        Key             : Value
                   Corn : 230.0
        Favorable_Beets : 260.0
          Regular_Beets : 260.0
                  Wheat : 150.0
    crop_yield : Size=4, Index=CROPS, Domain=Any, Default=None, Mutable=False
        Key             : Value
                   Corn :   3.0
        Favorable_Beets :  20.0
          Regular_Beets :  20.0
                  Wheat :   2.5
    max_possible : Size=4, Index=CROPS, Domain=Any, Default=None, Mutable=False
        Key             : Value
                   Corn :    nan
        Favorable_Beets : 6000.0
          Regular_Beets :    nan
                  Wheat :    nan
    min_required : Size=4, Index=CROPS, Domain=Any, Def